In [1]:
import re
from copy import copy

In [2]:
class Valve:
    def __init__(self, flow_rate, neighbors):
        self.flow_rate = flow_rate
        self.neighbors = neighbors

def read_input(filename):
    valves = {}
    expr = r'^Valve ([A-Z]{2}) has flow rate=(\d+); tunnels? leads? to valves? (.*)$'
    with open(filename, 'r') as f:
        for line in f.readlines():
            m = re.search(expr, line)
            flow_rate = int(m[2])
            neighbors = m[3].split(', ')
            valves[m[1]] = Valve(flow_rate, neighbors)
    return valves

def bfs(valves, start):
    q = [start]
    visited = set(start)
    dist = {start: 0}
    while q:
        u = q.pop(0)
        for v in valves[u].neighbors:
            if v not in visited:
                visited.add(v)
                dist[v] = dist[u] + 1
                q.append(v)
    return dist

def calc_dists(valves):
    flow_rate_valves = [v for v in valves if valves[v].flow_rate > 0]
    dists = {}
    for start_valve in ['AA'] + flow_rate_valves:
        d = bfs(valves, start_valve)
        dists[start_valve] = {k: v for (k, v) in d.items() if k in flow_rate_valves and not k == start_valve}
    return dists

def dfs(dists, valves, time, start, opened_valves):
    comb = [opened_valves]
    if time < 2:
        return comb
    for adj in valves:
        new_time   = time - dists[start][adj] - 1
        new_ov = opened_valves | {adj: new_time}
        new_valves = valves.difference({adj})
        comb += dfs(dists, new_valves, new_time, adj, new_ov)
    return comb

def runit(filename):
    valves = read_input(filename)
    dists = calc_dists(valves)
    flow_valves = {k for (k, v) in valves.items() if v.flow_rate > 0}
    comb = dfs(dists, flow_valves, 30, 'AA', {})
    max_pressure = 0
    for visited in comb:
        pressure = sum(valves[k].flow_rate * v for (k, v) in visited.items())
        if pressure > max_pressure:
            max_pressure = pressure
    return max_pressure

def runit2(filename):
    valves = read_input(filename)
    dists = calc_dists(valves)
    flow_valves = {k for (k, v) in valves.items() if v.flow_rate > 0}
    comb = dfs(dists, flow_valves, 26, 'AA', {})
    comb_sorted = sorted(comb, key=lambda x: sum(valves[k].flow_rate * v for (k, v) in x.items()), reverse=True)
    max_pressure = 0
    for human in comb_sorted:
        for elephant in comb_sorted:
            human_pressure = (sum(valves[k].flow_rate * v for (k, v) in human.items()))
            elephant_pressure = (sum(valves[k].flow_rate * v for (k, v) in elephant.items()))
            pressure = human_pressure + elephant_pressure
            if pressure <= max_pressure:
                break
            human_set = set(human.keys())
            elephant_set = set(elephant.keys())
            if len(human_set.intersection(elephant_set)) == 0:
                if pressure > max_pressure:
                    max_pressure = pressure
    return max_pressure


In [4]:
runit('16_input.txt')

KeyboardInterrupt: 

In [4]:
runit2('16_input.txt')

2469